<a href="https://colab.research.google.com/github/Sihan-A/Pydata_2e_Chinese_Notebook/blob/main/ch07_%E6%95%B0%E6%8D%AE%E6%B8%85%E6%B4%97.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/wesm/pydata-book.git
# 移到当前文件夹
%cd /content/pydata-book

Cloning into 'pydata-book'...
remote: Enumerating objects: 447, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 447 (delta 2), reused 6 (delta 2), pack-reused 440
Receiving objects: 100% (447/447), 40.55 MiB | 15.33 MiB/s, done.
Resolving deltas: 100% (139/139), done.
Checking out files: 100% (210/210), done.
/content/pydata-book


# 7 数据清理与准备

- 80%时间花在数据准备上
- 缺失值、重复值、字符串操作、其他分析数据转换工具

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
PREVIOUS_MAX_ROWS = pd.options.display.max_rows # 60
pd.options.display.max_rows = 20
np.random.seed(12345)
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

## 7.1 处理缺失值

缺失值`NaN`（Not a Number），容易检测到的标识值

In [ ]:
# 创建有缺失值的Series
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])
string_data

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [ ]:
string_data.isnull() # isnull()检测时候有缺失值，True→缺失值

0    False
1    False
2     True
3    False
dtype: bool

In [ ]:
string_data[0] = None # 将第0项改为None
string_data

0         None
1    artichoke
2          NaN
3      avocado
dtype: object

In [ ]:
string_data.isnull() # Python的None在Pandas中被当作NA处理

0     True
1    False
2     True
3    False
dtype: bool

**缺失值相关的方法**
- `dropna()`
- `fillna()`
- `isnull()`
- `notnull()`

### 过滤缺失值

In [ ]:
from numpy import nan as NA
data = pd.Series([1, NA, 3.5, NA, 7])
data

0    1.0
1    NaN
2    3.5
3    NaN
4    7.0
dtype: float64

In [ ]:
data.dropna() # 返回非空数据及index

0    1.0
2    3.5
4    7.0
dtype: float64

In [ ]:
data.notnull()

0     True
1    False
2     True
3    False
4     True
dtype: bool

In [ ]:
data[data.notnull()] # = data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

In [ ]:
data = pd.DataFrame([[1., 6.5, 3.], # 创建有NA的DataFrame
                     [1., NA, NA],
                     [NA, NA, NA],
                     [NA, 6.5, 3.]])
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [ ]:
cleaned = data.dropna() # dropna默认删除有NA的行
cleaned

,0,1,2
0,1.0,6.5,3.0


In [ ]:
data.dropna(how = 'all') # 删除所有元素是NA的行

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


In [ ]:
data[4] = NA # 添加所有元素是NA的列
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [ ]:
data.dropna(axis=1, # axis=1: 按列；
            how='all') # 删除所有元素是NA的行或列

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [ ]:
df = pd.DataFrame(np.random.randn(7, 3))
df.iloc[:4, 1] = NA
df.iloc[:2, 2] = NA
df

,0,1,2
0,-0.204708,NaN,NaN
1,-0.555730,NaN,NaN
2,0.092908,NaN,0.769023
3,1.246435,NaN,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [ ]:
df.dropna() # 删除有NA的行

,0,1,2
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


thresh=n：保留的行/列有 ≥ n个非NA值

In [ ]:
df.dropna(thresh=2) # 0行和1行只有1个非NA值，所有被drop掉了

,0,1,2
2,0.092908,NaN,0.769023
3,1.246435,NaN,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


### 补全缺失值
- 主要用`fillna`方法来补全缺失值

In [ ]:
df.fillna(0) # 用常数0代替NA

,0,1,2
0,-0.204708,0.000000,0.000000
1,-0.555730,0.000000,0.000000
2,0.092908,0.000000,0.769023
3,1.246435,0.000000,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [ ]:
# 使用字典时，不同列设定不同的填充值
df.fillna({1: 0.5, # 第1列用0.5填充NAN
           2: 0}) # 第2列用0填充NAN

,0,1,2
0,-0.204708,0.500000,0.000000
1,-0.555730,0.500000,0.000000
2,0.092908,0.500000,0.769023
3,1.246435,0.500000,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


`fillna`会返回一个新的对象，`inplace=True`时修改已经存在的对象

In [ ]:
# _表示不重要或临时的变量，经常用在unpacking中
_ = df.fillna(0, # NA → 0
              inplace=True) # 原地修改
df

,0,1,2
0,-0.204708,0.000000,0.000000
1,-0.555730,0.000000,0.000000
2,0.092908,0.000000,0.769023
3,1.246435,0.000000,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [ ]:
df = pd.DataFrame(np.random.randn(6, 3))
df.iloc[2:, 1] = NA
df.iloc[4:, 2] = NA
df

,0,1,2
0,0.476985,3.248944,-1.021228
1,-0.577087,0.124121,0.302614
2,0.523772,NaN,1.343810
3,-0.713544,NaN,-2.370232
4,-1.860761,NaN,NaN
5,-1.265934,NaN,NaN


In [ ]:
df.fillna(method='ffill') # forward fill

,0,1,2
0,0.476985,3.248944,-1.021228
1,-0.577087,0.124121,0.302614
2,0.523772,0.124121,1.343810
3,-0.713544,0.124121,-2.370232
4,-1.860761,0.124121,-2.370232
5,-1.265934,0.124121,-2.370232


In [ ]:
df.fillna(method='ffill', # forward fill，前面的值来填充NA
          limit=2) # ffill/bfill的最多数量

,0,1,2
0,0.476985,3.248944,-1.021228
1,-0.577087,0.124121,0.302614
2,0.523772,0.124121,1.343810
3,-0.713544,0.124121,-2.370232
4,-1.860761,NaN,-2.370232
5,-1.265934,NaN,-2.370232


In [ ]:
data = pd.Series([1., NA, 3.5, NA, 7])
data.fillna(data.mean()) # data的平均值来填充NA

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

In [ ]:
data.fillna(data.median()) # data的中位数来填充NA

0    1.0
1    3.5
2    3.5
3    3.5
4    7.0
dtype: float64

**`fillna`方法的参数**

| 参数      | 描述                                      |
| :--------- | :----------------------------------------- |
| `value`   | 标量值或字典型对象用于填充缺失值          |
| `method`  | 插值方法，如果没有其他参数，默认是`ffill` |
| `axis`    | 需要填充的轴，默认`axis=0`                |
| `inplace` | 修改被调用的对象，而不是生成一个备份      |
| `limit`   | 用于前向或后向填充时最大的填充范围        |

## 7.2 数据转换

### 7.2.1 删除重复值

In [ ]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                     'k2': [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [ ]:
data.duplicated() # duplicated返回一个布尔值Series，True表示与之前出现过的行重复
# 第6行True，表示它是重复行，与第5行重复

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [ ]:
data.drop_duplicates() # 返回FataFrame；duplicated中False的部分

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


In [ ]:
data['v1'] = range(7) # 加新列v1
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [ ]:
data.drop_duplicates(['k1']) # 基于k1列去除重复值

,k1,k2,v1
0,one,1,0
1,two,1,1


In [ ]:
data.drop_duplicates(['k1', 'k2'], # 基于k1和k2列去除重复值
                     keep='last') # keep='last'返回最后一个观测值，默认是返回第一个

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


### Transforming Data Using a Function or Mapping

In [ ]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                              'Pastrami', 'corned beef', 'Bacon',
                              'pastrami', 'honey ham', 'nova lox'],
                     'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [ ]:
# 表明食品的动物类型的字典
meat_to_animal = {'bacon': 'pig',
                  'pulled pork': 'pig',
                  'pastrami': 'cow',
                  'corned beef': 'cow',
                  'honey ham': 'pig',
                  'nova lox': 'salmon'}

In [ ]:
lowercased_food = data['food'].str.lower() # Series.str.lower()方法转为小写
lowercased_food

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [ ]:
lowercased_food.map(meat_to_animal) # map()方法接受函数或一个包含映射关系的字典型对象

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

In [ ]:
data['animal'] = lowercased_food.map(meat_to_animal) # 加新列
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [ ]:
data['food'].map(lambda x: meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

### Replacing Values 替代值

In [ ]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [ ]:
# replace方法：-999 → NaN
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [ ]:
# replace方法：-999和-1000 → NaN
data.replace([-999, -1000], np.nan) 

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [ ]:
data.replace([-999, -1000], # replace方法：-999 → NaN
             [np.nan, 0]) # -1000 → 0

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [ ]:
data.replace({-999: np.nan, # replace方法传字典：-999 → NaN
              -1000: 0}) # -1000 → 0

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

1. `data.replace()`
2. `data.str.replace()`：对字符串按元素替代

### Renaming Axis Indexes 重命名轴索引

In [ ]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [ ]:
# 定义一个lambda函数
transform = lambda x: x[:4].upper() # 返回x的切片[:4]的大写
data.index.map(transform) # data的index上用函数transform

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

In [ ]:
data.index = data.index.map(transform) # 重新定义index
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [ ]:
data.rename(index=str.title, # index首字母大写
            columns=str.upper) # column大写

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


In [ ]:
# rename配合字典
data.rename(index={'OHIO': 'INDIANA'}, # 'OHIO'行→'INDIANA'行
            columns={'three': 'peekaboo'}) # 'three'列→'peekaboo'列

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [ ]:
data.rename(index={'OHIO': 'INDIANA'}, # 'OHIO'行→'INDIANA'行
            inplace=True) # 原地修改
data

,one,two,three,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


### Discretization and Binning 离散化和分箱

- `pd.cut()`

In [ ]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32] # 年龄框
bins = [18, 25, 35, 60, 100]
cats = pd.cut(ages, bins)
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

- `codes`属性
- `categoties`属性

In [ ]:
cats.codes # codes属性

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [ ]:
cats.categories # categories属性；默认左边开放，右边封闭

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]],
              closed='right',
              dtype='interval[int64]')

In [ ]:
pd.value_counts(cats) # 对pd.cut结果中箱数量的计数

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

In [ ]:
pd.cut(ages, [18, 26, 36, 61, 100],
       right=False) # 右边开放；默认True

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

In [ ]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
cutted = pd.cut(ages, bins,
                labels=group_names) # 自定义箱子名
cutted

['Youth', 'Youth', 'Youth', 'YoungAdult', 'Youth', ..., 'YoungAdult', 'Senior', 'MiddleAged', 'MiddleAged', 'YoungAdult']
Length: 12
Categories (4, object): ['Youth' < 'YoungAdult' < 'MiddleAged' < 'Senior']

In [ ]:
pd.value_counts(cutted) # 计数

Youth         5
MiddleAged    3
YoungAdult    3
Senior        1
dtype: int64

In [ ]:
data = np.random.rand(20) # [0, 1)的均匀分布
pd.cut(data, # 传入数据
       4, # 4个等长的箱
       precision=2) # 科学计数法留两位

[(0.34, 0.55], (0.34, 0.55], (0.76, 0.97], (0.76, 0.97], (0.34, 0.55], ..., (0.34, 0.55], (0.34, 0.55], (0.55, 0.76], (0.34, 0.55], (0.12, 0.34]]
Length: 20
Categories (4, interval[float64]): [(0.12, 0.34] < (0.34, 0.55] < (0.55, 0.76] < (0.76, 0.97]]

- `pd.qcut()`

In [ ]:
data = np.random.randn(1000)  # 标准正态分布
cats = pd.qcut(data, 4)  # 根据样本的分位数，获得4个等长的箱，也就是每个箱子里有250个样本的分法
cats

[(-0.0265, 0.62], (0.62, 3.928], (-0.68, -0.0265], (0.62, 3.928], (-0.0265, 0.62], ..., (-0.68, -0.0265], (-0.68, -0.0265], (-2.9499999999999997, -0.68], (0.62, 3.928], (-0.68, -0.0265]]
Length: 1000
Categories (4, interval[float64]): [(-2.9499999999999997, -0.68] < (-0.68, -0.0265] < (-0.0265, 0.62] <
                                    (0.62, 3.928]]

In [ ]:
pd.value_counts(cats) # 计数

(0.62, 3.928]                   250
(-0.0265, 0.62]                 250
(-0.68, -0.0265]                250
(-2.9499999999999997, -0.68]    250
dtype: int64

In [ ]:
pd.qcut(data,
        [0, 0.1, 0.5, 0.9, 1.]) # 传入自定义的分位数，0和1之间的数据

[(-0.0265, 1.286], (-0.0265, 1.286], (-1.187, -0.0265], (-0.0265, 1.286], (-0.0265, 1.286], ..., (-1.187, -0.0265], (-1.187, -0.0265], (-2.9499999999999997, -1.187], (-0.0265, 1.286], (-1.187, -0.0265]]
Length: 1000
Categories (4, interval[float64]): [(-2.9499999999999997, -1.187] < (-1.187, -0.0265] < (-0.0265, 1.286] <
                                    (1.286, 3.928]]

### Detecting and Filtering Outliers 检测和过滤异常值

In [ ]:
data = pd.DataFrame(np.random.randn(1000, 4)) # 形状(1000,4)的标准正态分布的ndarray
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.049091,0.026112,-0.002544,-0.051827
std,0.996947,1.007458,0.995232,0.998311
min,-3.645860,-3.184377,-3.745356,-3.428254
25%,-0.599807,-0.612162,-0.687373,-0.747478
50%,0.047101,-0.013609,-0.022158,-0.088274
75%,0.756646,0.695298,0.699046,0.623331
max,2.653656,3.525865,2.735527,3.366626


In [ ]:
col = data[2] # data的第2列为data
col[np.abs(col) > 3] # col中绝对值大于3的值

41    -3.399312
136   -3.745356
Name: 2, dtype: float64

In [ ]:
data[(np.abs(data) > 3).any(1)]

,0,1,2,3
41,0.457246,-0.025907,-3.399312,-0.974657
60,1.951312,3.260383,0.963301,1.201206
136,0.508391,-0.196713,-3.745356,-1.520113
235,-0.242459,-3.056990,1.918403,-0.578828
258,0.682841,0.326045,0.425384,-3.428254
322,1.179227,-3.184377,1.369891,-1.074833
544,-3.548824,1.553205,-2.186301,1.277104
635,-0.578093,0.193299,1.397822,3.366626
782,-0.207434,3.525865,0.283070,0.544635
803,-3.645860,0.255475,-0.549574,-1.907459


In [ ]:
data[np.abs(data) > 3] = np.sign(data) * 3 # 大于3→3；小于-3→-3
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.050286,0.025567,-0.001399,-0.051765
std,0.992920,1.004214,0.991414,0.995761
min,-3.000000,-3.000000,-3.000000,-3.000000
25%,-0.599807,-0.612162,-0.687373,-0.747478
50%,0.047101,-0.013609,-0.022158,-0.088274
75%,0.756646,0.695298,0.699046,0.623331
max,2.653656,3.000000,2.735527,3.000000


`np.sign()`符号函数
- The sign function returns
    - `-1` if x < 0,
    - `0` if x==0,
    - `1` if x > 0. 
    - `NaN` is returned for nan inputs.

In [ ]:
np.sign(data).head()

,0,1,2,3
0,-1.0,1.0,-1.0,1.0
1,1.0,-1.0,1.0,-1.0
2,1.0,1.0,1.0,-1.0
3,-1.0,-1.0,1.0,-1.0
4,-1.0,1.0,-1.0,-1.0


### Permutation and Random Sampling 置换和随机采样

In [ ]:
df = pd.DataFrame(np.arange(5 * 4).reshape((5, 4)))
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [ ]:
sampler = np.random.permutation(5)
sampler

array([3, 1, 4, 2, 0])

In [ ]:
df.take(sampler)

,0,1,2,3
3,12,13,14,15
1,4,5,6,7
4,16,17,18,19
2,8,9,10,11
0,0,1,2,3


In [ ]:
df.sample(n=3)

,0,1,2,3
3,12,13,14,15
4,16,17,18,19
2,8,9,10,11


In [ ]:
choices = pd.Series([5, 7, -1, 6, 4])
draws = choices.sample(n=10, replace=True)
draws

4    4
1    7
4    4
2   -1
0    5
3    6
1    7
4    4
0    5
4    4
dtype: int64

### Computing Indicator/Dummy Variables

In [ ]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                   'data1': range(6)})
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [ ]:
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [ ]:
dummies = pd.get_dummies(df['key'], prefix='key')
dummies

,key_a,key_b,key_c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [ ]:
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


In [ ]:
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('./datasets/movielens/movies.dat',
                        sep='::',
                        engine='python',
                        header=None,
                        names=mnames)
movies[:10]

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [ ]:
all_genres = [] # an empty list
for x in movies.genres:
    all_genres.extend(x.split('|'))
genres = pd.unique(all_genres)
genres

array(['Animation', "Children's", 'Comedy', 'Adventure', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Sci-Fi', 'Documentary', 'War', 'Musical', 'Mystery', 'Film-Noir',
       'Western'], dtype=object)

In [ ]:
len(movies)

3883

In [ ]:
len(genres)

18

In [ ]:
zero_matrix = np.zeros((len(movies), len(genres))) # 3883 * 18 zero matrix
dummies = pd.DataFrame(zero_matrix,
                       columns=genres)

In [ ]:
gen = movies.genres[0]
gen.split('|')

['Animation', "Children's", 'Comedy']

In [ ]:
dummies.columns.get_indexer(gen.split('|'))

array([0, 1, 2])

In [ ]:
for i, gen in enumerate(movies.genres):
    indices = dummies.columns.get_indexer(gen.split('|'))
    dummies.iloc[i, indices] = 1

In [ ]:
movies_windic = movies.join(dummies.add_prefix('Genre_'))
movies_windic.iloc[0]

movie_id                                      1
title                          Toy Story (1995)
genres              Animation|Children's|Comedy
Genre_Animation                               1
Genre_Children's                              1
                               ...             
Genre_War                                     0
Genre_Musical                                 0
Genre_Mystery                                 0
Genre_Film-Noir                               0
Genre_Western                                 0
Name: 0, Length: 21, dtype: object

In [ ]:
np.random.seed(12345)
values = np.random.rand(10)
values

array([0.9296, 0.3164, 0.1839, 0.2046, 0.5677, 0.5955, 0.9645, 0.6532,
       0.7489, 0.6536])

In [ ]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,0,0,0,0,1
1,0,1,0,0,0
2,1,0,0,0,0
3,0,1,0,0,0
4,0,0,1,0,0
5,0,0,1,0,0
6,0,0,0,0,1
7,0,0,0,1,0
8,0,0,0,1,0
9,0,0,0,1,0


## String Manipulation

### 7.3.1 字符串对象方法

`split()`

In [ ]:
val = 'a,b, guido'
val.split(',')

['a', 'b', ' guido']

`strip()`和`split()`

In [ ]:
pieces = [x.strip() for x in val.split(',')]
pieces

['a', 'b', 'guido']

In [ ]:
first, second, third = pieces
first + '::' + second + '::' + third

'a::b::guido'

In [ ]:
'::'.join(pieces)

'a::b::guido'

In [ ]:
'guido' in val

True

In [ ]:
val.index(',')

1

In [ ]:
val.find(':')

-1

```python
>>> val.index(':')
-------------------------------------------------------
ValueError Traceback (most recent call last)
<ipython-input-283-2c016e7367ac> in <module>()
----> 1 val.index(':')
ValueError: substring not found
```

In [ ]:
val.count(',')

2

In [ ]:
val.replace(',', '::')

'a::b:: guido'

In [ ]:
val.replace(',', '')

'ab guido'

### Regular Expressions

In [ ]:
import re
text = "foo    bar\t baz  \tqux"
re.split('\s+', text)

['foo', 'bar', 'baz', 'qux']

In [ ]:
regex = re.compile('\s+')
regex.split(text)

['foo', 'bar', 'baz', 'qux']

In [ ]:
regex.findall(text)

['    ', '\t ', '  \t']

In [ ]:
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""
pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'

# re.IGNORECASE makes the regex case-insensitive
regex = re.compile(pattern, flags=re.IGNORECASE)

In [ ]:
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

In [ ]:
m = regex.search(text)
m

<re.Match object; span=(5, 20), match='dave@google.com'>

In [ ]:
text[m.start():m.end()]

'dave@google.com'

In [ ]:
print(regex.match(text))

None


In [ ]:
print(regex.sub('REDACTED', text))

Dave REDACTED
Steve REDACTED
Rob REDACTED
Ryan REDACTED



In [ ]:
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'
regex = re.compile(pattern, flags=re.IGNORECASE)

In [ ]:
m = regex.match('wesm@bright.net')
m.groups()

('wesm', 'bright', 'net')

In [ ]:
regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

In [ ]:
print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text))

Dave Username: dave, Domain: google, Suffix: com
Steve Username: steve, Domain: gmail, Suffix: com
Rob Username: rob, Domain: gmail, Suffix: com
Ryan Username: ryan, Domain: yahoo, Suffix: com



### 矢量化字符串函数

In [ ]:
data = {'Dave' : 'dave@google.com',
        'Steve': 'steve@gmail.com',
        'Rob'  : 'rob@gmail.com',
        'Wes'  :  np.nan}
data = pd.Series(data)
data

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

In [ ]:
data.isnull()

Dave     False
Steve    False
Rob      False
Wes       True
dtype: bool

In [ ]:
data.str.contains('gmail')

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

In [ ]:
pattern

'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\\.([A-Z]{2,4})'

In [ ]:
data.str.findall(pattern, flags = re.IGNORECASE)

Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

In [ ]:
matches = data.str.match(pattern,
                         flags = re.IGNORECASE)
matches

Dave     True
Steve    True
Rob      True
Wes       NaN
dtype: object

In [ ]:
matches.str.get(1)

AttributeError: ignored

In [ ]:
matches.str[0]

In [ ]:
data.str[:5]

In [ ]:
pd.options.display.max_rows = PREVIOUS_MAX_ROWS

## Conclusion